In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import time
import warnings
import os

# Scikit-learn
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor

# XGBoost
import xgboost as xgb

# Cấu hình hiển thị
pd.set_option('display.float_format', lambda x: '%.4f' % x)
warnings.filterwarnings('ignore')

# Kiểm tra GPU
import tensorflow as tf
print("🔥 GPU Status:", tf.config.list_physical_devices('GPU'))
print("✅ Sẵn sàng chiến đấu với 75k dòng dữ liệu!")

🔥 GPU Status: []
✅ Sẵn sàng chiến đấu với 75k dòng dữ liệu!


In [19]:
# Đường dẫn file mới (73k dòng)
DATA_PATH = '../data/processed/clean_vn_housing.csv'

df = pd.read_csv(DATA_PATH)
print(f"📂 Đã load dữ liệu: {df.shape}")
# Kỳ vọng: (73652, 358)

# 1. Tách Feature & Target
target_col = 'Total_Price_Billion'

# Loại bỏ các cột không dùng để train
# Lưu ý: File sạch đã bỏ Address và Price_per_m2, nhưng ta cứ drop lần nữa cho chắc
drop_cols = [target_col, 'Price_per_m2', 'Address', 'Unnamed: 0']
X = df.drop(columns=[c for c in drop_cols if c in df.columns], errors='ignore')

# 2. Log Transform Target
# BẮT BUỘC: Giúp model dự đoán tốt dải giá rộng từ 1 tỷ đến 100 tỷ
y = np.log1p(df[target_col])

# 3. Chia tập Train/Test (80/20)
# Với 73k dòng, tập test sẽ có khoảng 14.7k dòng -> Rất đủ để đánh giá tin cậy
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. Scaling (RobustScaler)
# Cực quan trọng khi có cột Length/Width/Area biên độ chênh lệch lớn
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Lưu Scaler cho App
if not os.path.exists('../models'): os.makedirs('../models')
joblib.dump(scaler, '../models/scaler_final.pkl')

print(f"✅ Dữ liệu đã chuẩn hóa. Số Features: {X_train.shape[1]}")

📂 Đã load dữ liệu: (65468, 346)
✅ Dữ liệu đã chuẩn hóa. Số Features: 345


In [20]:
def danh_gia_hieu_nang(model, X_test, y_test_log, ten_mo_hinh):
    # Dự đoán
    y_pred_log = model.predict(X_test)

    # Chuyển ngược Log -> Giá thực (Tỷ VNĐ)
    y_true = np.expm1(y_test_log)
    y_pred = np.expm1(y_pred_log)
    y_pred = np.maximum(y_pred, 0) # Chặn số âm

    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)

    print(f"\n{'='*40}")
    print(f"📊 KẾT QUẢ: {ten_mo_hinh.upper()}")
    print(f"{'='*40}")
    print(f"MAE  : {mae:.4f} Tỷ")
    print(f"RMSE : {rmse:.4f} Tỷ")
    print(f"R2   : {r2:.4f} ({r2*100:.2f}%)")

    if r2 > 0.7: print("   🚀 Model đã đạt mức độ chính xác cao!")
    elif r2 > 0.8: print("   🏆 Model xuất sắc (SOTA)!")

    return {'Model': ten_mo_hinh, 'MAE': mae, 'RMSE': rmse, 'R2': r2}

In [21]:
print(">>> 1. Tuning Random Forest (Big Data Optimized)...")
t0 = time.time()

rf = RandomForestRegressor(random_state=42, n_jobs=-1)

param_grid_rf = {
   'n_estimators': [400, 500],    # Số lượng cây lớn để ổn định
    'max_depth': [25, 30],         # Giới hạn độ sâu vừa phải
    'max_features': [0.5, 'sqrt'], # Nhìn 50% số cột để nắm bắt Tên đường tốt hơn

    # QUAN TRỌNG: min_samples_leaf = 2
    # Bắt buộc mỗi nhánh cuối cùng phải có ít nhất 2 căn nhà.
    # Điều này ngăn model "nhớ vẹt" một căn nhà duy nhất (Chống Overfitting hiệu quả nhất).
    'min_samples_leaf': [2],
    'min_samples_split': [5]
}

# Dùng n_jobs=-1 để ép CPU chạy 100% công suất
grid_rf = GridSearchCV(rf, param_grid_rf, cv=3, scoring='r2', verbose=1, n_jobs=-1)
grid_rf.fit(X_train_scaled, y_train)

best_rf = grid_rf.best_estimator_
print(f"⏱️ RF Time: {time.time() - t0:.1f}s")
print("Best Params RF:", grid_rf.best_params_)

# Lưu và đánh giá
joblib.dump(best_rf, '../models/rf_final.pkl')
res_rf = danh_gia_hieu_nang(best_rf, X_test_scaled, y_test, "Random Forest (Full Data)")

>>> 1. Tuning Random Forest (Big Data Optimized)...
Fitting 3 folds for each of 8 candidates, totalling 24 fits
⏱️ RF Time: 747.9s
Best Params RF: {'max_depth': 30, 'max_features': 0.5, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 500}

📊 KẾT QUẢ: RANDOM FOREST (FULL DATA)
MAE  : 0.6049 Tỷ
RMSE : 1.0079 Tỷ
R2   : 0.8366 (83.66%)
   🚀 Model đã đạt mức độ chính xác cao!


In [22]:
train_r2 = r2_score(y_train, best_rf.predict(X_train_scaled))
test_r2  = r2_score(y_test, best_rf.predict(X_test_scaled))

print("Train R2:", train_r2)
print("Test  R2:", test_r2)
print("Gap     :", train_r2 - test_r2)

Train R2: 0.8928784011454947
Test  R2: 0.8174614186143531
Gap     : 0.0754169825311416


In [23]:
print(">>> 2. Tuning XGBoost (GPU RTX 3050 Power)...")
t0 = time.time()

# Cấu hình chuẩn cho GPU
xgb_model = xgb.XGBRegressor(
    objective='reg:squarederror',
    tree_method='hist', # Bắt buộc cho dữ liệu lớn (>50k dòng)
    device='cuda',      # Kích hoạt GPU
    random_state=42,
    n_jobs=1
)

param_grid_xgb = {
   'n_estimators': [3000],          # 3000 cây là đủ cho dữ liệu này
    'learning_rate': [0.01],         # Học chậm để chắc chắn
    'max_depth': [10, 12],           # Độ sâu đủ để hiểu Tên đường

    # CHỐNG OVERFITTING:
    'reg_alpha': [1.0, 2.0],         # Tăng phạt L1 (Cũ là 0.1 -> Giờ tăng lên 1.0)
    'reg_lambda': [5.0],             # Phạt L2
    'subsample': [0.85],             # Lấy mẫu ngẫu nhiên 85%
    'colsample_bytree': [0.6]        # Nhìn 60% số cột
}

grid_xgb = GridSearchCV(xgb_model, param_grid_xgb, cv=3, scoring='r2', verbose=1, n_jobs=1)
grid_xgb.fit(X_train_scaled, y_train)

best_xgb = grid_xgb.best_estimator_
print(f"⏱️ XGB Time: {time.time() - t0:.1f}s")
print("Best Params XGB:", grid_xgb.best_params_)

# Lưu model (Model này sẽ dùng cho App)
res_xgb = danh_gia_hieu_nang(best_xgb, X_test_scaled, y_test, "XGBoost (Full Data)")
joblib.dump(best_xgb, '../models/xgb_final.pkl')

>>> 2. Tuning XGBoost (GPU RTX 3050 Power)...
Fitting 3 folds for each of 4 candidates, totalling 12 fits
⏱️ XGB Time: 558.6s
Best Params XGB: {'colsample_bytree': 0.6, 'learning_rate': 0.01, 'max_depth': 12, 'n_estimators': 3000, 'reg_alpha': 1.0, 'reg_lambda': 5.0, 'subsample': 0.85}

📊 KẾT QUẢ: XGBOOST (FULL DATA)
MAE  : 0.6045 Tỷ
RMSE : 1.0191 Tỷ
R2   : 0.8329 (83.29%)
   🚀 Model đã đạt mức độ chính xác cao!


['../models/xgb_final.pkl']

In [24]:
train_r2 = r2_score(y_train, best_xgb.predict(X_train_scaled))
test_r2  = r2_score(y_test, best_xgb.predict(X_test_scaled))

print("Train R2:", train_r2)
print("Test  R2:", test_r2)
print("Gap     :", train_r2 - test_r2)

Train R2: 0.8555807924910348
Test  R2: 0.8211855216874435
Gap     : 0.03439527080359128


In [26]:
import pandas as pd

# 1. Gom kết quả từ 3 model (Đảm bảo bạn đã chạy xong 3 cell trên)
results_list = [res_rf, res_xgb]

# 2. Tạo bảng DataFrame
df_results = pd.DataFrame(results_list)

# 3. Sắp xếp theo R2 giảm dần (Model xịn nhất lên đầu)
df_results = df_results.sort_values(by='R2', ascending=False).reset_index(drop=True)

# 4. Hiển thị bảng đơn giản
print(">>> BẢNG SO SÁNH KẾT QUẢ CUỐI CÙNG")
display(df_results)

# (Tùy chọn) Lưu ra file Excel/CSV để nộp báo cáo
df_results.to_csv('../data/processed/ket_qua_so_sanh.csv', index=False)

>>> BẢNG SO SÁNH KẾT QUẢ CUỐI CÙNG


,Model,MAE,RMSE,R2
0,Random Forest (Full Data),0.6049,1.0079,0.8366
1,XGBoost (Full Data),0.6045,1.0191,0.8329
